In [2]:
import pandas as pd
import numpy as np
import pydot as pydot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.express as px
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12,6)
from sklearn.model_selection import train_test_split

In [11]:
path = "//geo-file/geomatic/Temp/mao/Thesis/Data/"
file = "20231804_enriched_data_avg.csv"
df = pd.read_csv(path+file, sep=";")
df = df.dropna().reset_index(drop=True)
df.shape

(164335, 34)

In [5]:
df.head()

,UNADR_KEY,HOU_HOLHOMOWNR,MUNICIPALITY,CNZTYP,CNZGRP,OWNSHIP,USAGE,AREA,ROOMS,BUSINESS_ON_ADRESS,...,HOU_INCO_AVG,MARSTA_COUPLE,MARSTA_SINGLE,MARSTA_OTH,SOCGRP_AVG,WEALTH_AVG,PER_INCO_AVG,AGE_OLDST_AVG,CHL_AVG,Target
0,20000021,False,101,E6,E,PUB_RENT,Apartment,59.0,2.0,False,...,349153.0,0.000018,0.850385,0.149597,2.839109,-207.0,324791.0,51.33,0.333333,1
1,20000034,False,101,E3,E,PUB_RENT,Apartment,59.0,2.0,False,...,291456.0,0.000018,0.850385,0.149597,3.844031,1209355.0,291709.0,65.31,0.000289,1
2,20000035,False,101,E3,E,PUB_RENT,Apartment,68.0,3.0,True,...,326973.0,0.166667,0.699005,0.134328,2.947761,552221.0,290098.0,72.09,0.000289,1
3,20000041,False,101,E5,E,PUB_RENT,Apartment,31.0,1.0,True,...,200981.0,0.000018,0.850385,0.149597,3.360696,277497.0,201369.0,49.11,0.000289,0
4,20000048,False,101,F2,F,PUB_RENT,Apartment,79.0,4.0,False,...,1038600.0,0.857143,0.061576,0.081281,1.881773,1158497.0,662774.0,42.71,1.264339,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164335 entries, 0 to 164334
Data columns (total 34 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   UNADR_KEY           164335 non-null  int64  
 1   HOU_HOLHOMOWNR      164335 non-null  bool   
 2   MUNICIPALITY        164335 non-null  int64  
 3   CNZTYP              164335 non-null  object 
 4   CNZGRP              164335 non-null  object 
 5   OWNSHIP             164335 non-null  object 
 6   USAGE               164335 non-null  object 
 7   AREA                164335 non-null  float64
 8   ROOMS               164335 non-null  float64
 9   BUSINESS_ON_ADRESS  164335 non-null  bool   
 10  CAR_AVG             164335 non-null  float64
 11  EDU_AVG             164335 non-null  float64
 12  EMPL_SELF           164335 non-null  float64
 13  EMPL_TOP            164335 non-null  float64
 14  EMPL_HIGH           164335 non-null  float64
 15  EMPL_MED            164335 non-nul

In [7]:
df['HOU_HOLHOMOWNR'] = (df['HOU_HOLHOMOWNR']==True).astype(int)
df['BUSINESS_ON_ADRESS'] = (df['BUSINESS_ON_ADRESS']==True).astype(int)
df['Target'] = df.Target.astype(int)
df= df.drop(['CNZGRP', 'UNADR_KEY', 'PER_INCO_AVG'], axis=1)

# Functions for regrouping variables

**Employment**
1 is employed, 0 is unemployed

In [14]:
def employment(df):
    employed = df['EMPL_SELF']+df['EMPL_TOP']+df['EMPL_HIGH']+df['EMPL_MED']+df['EMPL_BASE']+df['EMPL_OTHER']
    unemployed = df['EMPL_UNEMPL']+df['EMPL_STUDENT']+df['EMPL_RETRMT_EARLY']+df['EMPL_PENSNR']+df['EMPL_CHL']+df['EMPL_OTH']
    df.loc[employed>unemployed, 'EMPL_STATUS']=1
    df.loc[employed<=unemployed, 'EMPL_STATUS']=0
    df = df.drop(columns=['EMPL_SELF', 'EMPL_TOP', 'EMPL_HIGH', 'EMPL_MED', 'EMPL_BASE', 'EMPL_OTHER', 'EMPL_UNEMPL', 'EMPL_STUDENT', 'EMPL_RETRMT_EARLY', 'EMPL_PENSNR', 'EMPL_CHL', 'EMPL_OTH'])
    df['EMPL_STATUS'] = df.EMPL_STATUS.astype('category')
    return df

**Child and car**
rounded to 2 decimals because RF seems to have trouble distinguinshing the decimals

In [15]:
def child_car(df):
    df['CHL_AVG'] = round(df['CHL_AVG'],2)
    df['CAR_AVG'] = round(df['CAR_AVG'],2)
    return df

**Marital status**
1 is married, 0 is not married

In [16]:
def mari_status(df):
    df.loc[df['MARSTA_COUPLE']>(df['MARSTA_SINGLE']+df['MARSTA_OTH']), 'MARI_STATUS']=1
    df.loc[df['MARSTA_COUPLE']<=(df['MARSTA_SINGLE']+df['MARSTA_OTH']), 'MARI_STATUS']=0
    df = df.drop(columns=['MARSTA_COUPLE', 'MARSTA_SINGLE', 'MARSTA_OTH'])
    df['MARI_STATUS'] = df.MARI_STATUS.astype('category')
    return df

**Ownship**
1 = owned, 0 = not owned

In [17]:
def ownship(df):
    df['OWNSHIP'] = df.OWNSHIP.astype(str)
    df.loc[(df['OWNSHIP']=="PART") | (df['OWNSHIP'] == "OWN"), 'OWNSHIP']=1
    df.loc[(df['OWNSHIP']=="PRI_RENT") | (df['OWNSHIP'] == "PUB_RENT") | (df['OWNSHIP'] == "UNK"), 'OWNSHIP'] = 0
    df['OWNSHIP'] = df.OWNSHIP.astype('category')
    return df

**Rooms**
Remove observations with rooms over 9 that are not a farmhouse nor an institution. Consider feature engineering institutions.

In [29]:
def rooms(df):
 df = df.loc[~((df['ROOMS']>9) & (df['USAGE']!='Farmhouse') & (df['USAGE']!='Institution') & (df['BUSINESS_ON_ADRESS']==False))]
 return df

**Municipality**

In [19]:
def muni(df):
    good = df.loc[df['Target']==1].groupby(['MUNICIPALITY'])['MUNICIPALITY'].count()/df.loc[df['Target']==1].count()[0]
    bad = df.loc[df['Target']==0].groupby(['MUNICIPALITY'])['MUNICIPALITY'].count()/df.loc[df['Target']==0].count()[0]
    index = (bad/good)*100
    index = index.to_numpy()
    index[92] = 100 #0 bad customers in this municipality and only 11 observations in total so we set it to 1. It is on Læsø

    for i in np.arange(index.size):
        if index[i]>150:
            index[i] = 5
        elif (index[i]<150 and index[i]>=120):
            index[i] = 4
        elif (index[i]<120 and index[i]>=100):
            index[i] = 3
        elif (index[i]<100 and index[i]>=80):
            index[i] = 2
        elif (index[i]<80 and index[i]>=50):
            index[i] = 1
        elif index[i] < 50:
            index[i] = 0
    print(index)

    index_df = pd.DataFrame({'Index':index, 'MUNICIPALITY':list(good.index.values)})

    df['MUNICIPALITY'] = df.MUNICIPALITY.astype('int32')

    for i in np.arange(index_df.shape[0]):
        df.loc[(df['MUNICIPALITY']==index_df['MUNICIPALITY'][i]), 'MUNICIPALITY'] = index_df['Index'][i]

    df['MUNICIPALITY'] = df.MUNICIPALITY.astype('category')
    return df

# Create different versions of training data
We will test different levels of feature engineering from baseline/none to more engineered
These dataframes will be saved as csv files for easier future use.
The data will be shuffled and split into train and test before being saved
Pre-processing is still included as it is a requirement for model to work (i.e. model cannot take "Etagebolig", it needs to be encoded)
The lowest data complexity level will only yield the absolute necessary steps for inputting to models

*Level 0*

In [9]:
df_0 = pd.get_dummies(df, columns=['USAGE','CNZGRP','MUNICIPALITY', 'OWNSHIP'])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_0.loc[:, ~df_0.columns.isin(['Target'])], df_0.loc[:,df_0.columns=='Target'], shuffle=True, test_size=0.25,
                                                 random_state=1, stratify=df_0.loc[:,df_0.columns=='Target'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(131468, 148) (32867, 148) (131468, 1) (32867, 1)


In [14]:
X_train.head()

,HOU_HOLHOMOWNR,AREA,ROOMS,BUSINESS_ON_ADRESS,CAR_AVG,EDU_AVG,EMPL_SELF,EMPL_TOP,EMPL_HIGH,EMPL_MED,...,MUNICIPALITY_840,MUNICIPALITY_846,MUNICIPALITY_849,MUNICIPALITY_851,MUNICIPALITY_860,OWNSHIP_OWN,OWNSHIP_PART,OWNSHIP_PRI_RENT,OWNSHIP_PUB_RENT,OWNSHIP_UNK
144495,0,42.0,2.0,0,0.636139,13.19,0.027160,0.024691,0.246914,0.079012,...,0,0,0,0,0,1,0,0,0,0
78178,0,95.0,4.0,0,1.661391,12.33,0.115000,0.037500,0.175000,0.095000,...,0,0,0,0,0,1,0,0,0,0
5953,0,61.0,2.0,0,0.362963,12.33,0.042079,0.004950,0.126238,0.049505,...,0,0,0,0,0,0,0,0,1,0
53813,0,157.0,5.0,0,1.551663,16.92,0.093826,0.140741,0.432099,0.118519,...,0,0,0,0,0,1,0,0,0,0
145824,0,106.0,4.0,0,0.745679,15.67,0.062189,0.027363,0.462687,0.087065,...,0,0,0,0,0,0,0,0,1,0


In [20]:
X_train.to_pickle('x_train_lvl0.pkl')
X_test.to_pickle('x_test_lvl0.pkl')
y_train.to_pickle('y_train_lvl0.pkl')
y_test.to_pickle('y_test_lvl0.pkl')

# Level 1
Categorical data is feature engineered and most dominant outliers are removed
Reduces to 64 variables

**Fixing data types**

In [30]:
df_3 = df

In [31]:
df_3['HOU_HOLHOMOWNR'] = (df_3['HOU_HOLHOMOWNR']==True).astype(int)
df_3['BUSINESS_ON_ADRESS'] = (df_3['BUSINESS_ON_ADRESS']==True).astype(int)
df_3['Target'] = df.Target.astype(int)
df_3= df_3.drop(['CNZGRP', 'UNADR_KEY', 'PER_INCO_AVG'], axis=1)

In [32]:
df_3 = employment(df_3)
df_3 = child_car(df_3)
df_3 = mari_status(df_3)
df_3 = ownship(df_3)
df_3 = rooms(df_3)
#df_3 = muni(df_3)

In [124]:
df_3 = pd.get_dummies(df_3, columns=['USAGE','CNZTYP','MUNICIPALITY'])

In [128]:
df_3['AREA'] = df_3.AREA.astype(int)
df_3['ROOMS'] = df_3.ROOMS.astype(int)
df_3['HOU_INCO_AVG'] = df_3.HOU_INCO_AVG.astype(int)
df_3['WEALTH_AVG'] = df_3.WEALTH_AVG.astype(int)
df_3['EMPL_STATUS'] = df_3.EMPL_STATUS.astype(int)
df_3['MARI_STATUS'] = df_3.MARI_STATUS.astype(int)
df_3['OWNSHIP'] = df_3.OWNSHIP.astype(int)
df_3['HOU_HOLHOMOWNR'] = (df_3['HOU_HOLHOMOWNR'] == True).astype(int)
df_3['BUSINESS_ON_ADRESS'] = (df_3['BUSINESS_ON_ADRESS'] == True).astype(int)
df_3['Target'] = df_3.Target.astype(int)
df_3.head()

,HOU_HOLHOMOWNR,OWNSHIP,AREA,ROOMS,BUSINESS_ON_ADRESS,PER_COU_UNADR,CAR_AVG,EDU_AVG,HOU_INCO_AVG,SOCGRP_AVG,...,CNZTYP_H1,CNZTYP_H2,CNZTYP_I1,CNZTYP_I2,CNZTYP_I3,CNZTYP_I4,MUNICIPALITY_1,MUNICIPALITY_2,MUNICIPALITY_3,MUNICIPALITY_4
0,0,0,59,2,0,1.4583,0.00,12.67,251903,3.281095,...,0,0,0,0,0,0,0,0,1,0
1,0,0,59,2,0,2.2157,0.18,13.67,277590,3.169154,...,0,0,0,0,0,0,0,0,1,0
2,0,0,68,3,1,2.2157,0.32,14.29,296572,3.002463,...,0,0,0,0,0,0,0,0,1,0
3,0,0,31,1,1,2.2157,0.00,11.67,221745,3.845905,...,0,0,0,0,0,0,0,0,1,0
4,0,0,79,4,0,2.2157,0.48,13.46,564137,3.002463,...,0,0,0,0,0,0,0,0,1,0


In [130]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Target'])],
                                                    df.loc[:, df.columns == 'Target'], shuffle=True, test_size=0.25,
                                                    random_state=1, stratify=df.loc[:, df.columns == 'Target'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train.to_pickle('x_train_lvl2.pkl')
X_test.to_pickle('x_test_lvl2.pkl')
y_train.to_pickle('y_train_lvl2.pkl')
y_test.to_pickle('y_test_lvl2.pkl')

(130205, 65) (32552, 65) (130205, 1) (32552, 1)


In [35]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Target'])],
                                                    df.loc[:, df.columns == 'Target'], shuffle=True, test_size=0.25,
                                                    random_state=1, stratify=df.loc[:, df.columns == 'Target'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(131144, 33) (32787, 33) (131144, 1) (32787, 1)


KeyError: 'Target'

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Target'])],
                                                    df.loc[:, df.columns == 'Target'], shuffle=True, test_size=0.2,
                                                    random_state=42, stratify=df.loc[:, df.columns == 'Target'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train.to_pickle('x_train_lvl3.pkl')
X_test.to_pickle('x_test_lvl3.pkl')
y_train.to_pickle('y_train_lvl3.pkl')
y_test.to_pickle('y_test_lvl3.pkl')

(131144, 65) (32787, 65) (131144, 1) (32787, 1)


# Lvl 3 - inclusive income per person


In [44]:
df['HOU_HOLHOMOWNR'] = (df['HOU_HOLHOMOWNR']==True).astype(int)
df['BUSINESS_ON_ADRESS'] = (df['BUSINESS_ON_ADRESS']==True).astype(int)
df['Target'] = df.Target.astype(int)
df= df.drop(['CNZGRP', 'UNADR_KEY', 'PER_INCO_AVG'], axis=1)

In [45]:
df = employment(df)
df = child(df)
df = mari_status(df)
df = ownship(df)
df = muni(df)

[3. 2. 4. 3. 1. 1. 2. 3. 2. 4. 3. 3. 2. 3. 4. 2. 2. 2. 1. 2. 3. 2. 1. 2.
 1. 2. 2. 3. 3. 2. 1. 1. 3. 2. 3. 3. 3. 4. 3. 4. 1. 4. 4. 4. 3. 3. 3. 2.
 2. 2. 3. 3. 3. 2. 3. 4. 4. 2. 3. 4. 3. 1. 3. 2. 4. 3. 2. 1. 1. 1. 3. 4.
 4. 2. 2. 1. 1. 3. 3. 2. 1. 2. 1. 2. 1. 4. 3. 2. 3. 1. 2. 4. 3. 1. 3. 2.
 2. 2.]


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163132 entries, 0 to 163131
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   UNADR_KEY           163132 non-null  int64   
 1   HOU_HOLHOMOWNR      163132 non-null  int32   
 2   MUNICIPALITY        163132 non-null  category
 3   CNZTYP              163132 non-null  object  
 4   OWNSHIP             163132 non-null  category
 5   USAGE               163132 non-null  object  
 6   AREA                163132 non-null  float64 
 7   ROOMS               163132 non-null  float64 
 8   BUSINESS_ON_ADRESS  163132 non-null  int32   
 9   PER_COU_UNADR       163132 non-null  float64 
 10  CAR_AVG             163132 non-null  float64 
 11  EDU_AVG             163132 non-null  float64 
 12  HOU_INCO_AVG        163132 non-null  float64 
 13  SOCGRP_AVG          163132 non-null  float64 
 14  WEALTH_AVG          163132 non-null  float64 
 15  AGE_OLDST_AVG    

In [47]:
df = df.loc[~((df['ROOMS'] > 9) & (df['USAGE'] != 'Farmhouse') & (df['USAGE'] != 'Institution') & (
            df['BUSINESS_ON_ADRESS'] == False))]
df = pd.get_dummies(df, columns=['USAGE', 'CNZTYP', 'MUNICIPALITY'])
df['AREA'] = df.AREA.astype(int)
df['ROOMS'] = df.ROOMS.astype(int)
df['HOU_INCO_AVG'] = df.HOU_INCO_AVG.astype(int)
df['WEALTH_AVG'] = df.WEALTH_AVG.astype(int)
df['EMPL_STATUS'] = df.EMPL_STATUS.astype(int)
df['MARI_STATUS'] = df.MARI_STATUS.astype(int)
df['OWNSHIP'] = df.OWNSHIP.astype(int)
df['HOU_HOLHOMOWNR'] = (df['HOU_HOLHOMOWNR'] == True).astype(int)
df['BUSINESS_ON_ADRESS'] = (df['BUSINESS_ON_ADRESS'] == True).astype(int)
df['Target'] = df.Target.astype(int)
df.head()

,UNADR_KEY,HOU_HOLHOMOWNR,OWNSHIP,AREA,ROOMS,BUSINESS_ON_ADRESS,PER_COU_UNADR,CAR_AVG,EDU_AVG,HOU_INCO_AVG,...,CNZTYP_H1,CNZTYP_H2,CNZTYP_I1,CNZTYP_I2,CNZTYP_I3,CNZTYP_I4,MUNICIPALITY_1,MUNICIPALITY_2,MUNICIPALITY_3,MUNICIPALITY_4
0,20000021,0,0,59,2,0,1.4583,0.00,12.67,251903,...,0,0,0,0,0,0,0,0,1,0
1,20000034,0,0,59,2,0,2.2157,0.18,13.67,277590,...,0,0,0,0,0,0,0,0,1,0
2,20000035,0,0,68,3,1,2.2157,0.32,14.29,296572,...,0,0,0,0,0,0,0,0,1,0
3,20000041,0,0,31,1,1,2.2157,0.00,11.67,221745,...,0,0,0,0,0,0,0,0,1,0
4,20000048,0,0,79,4,0,2.2157,0.48,13.46,564137,...,0,0,0,0,0,0,0,0,1,0


In [36]:
df.head()

,HOU_HOLHOMOWNR,OWNSHIP,AREA,ROOMS,BUSINESS_ON_ADRESS,PER_COU_UNADR,CAR_AVG,EDU_AVG,HOU_INCO_AVG,SOCGRP_AVG,...,CNZTYP_H1,CNZTYP_H2,CNZTYP_I1,CNZTYP_I2,CNZTYP_I3,CNZTYP_I4,MUNICIPALITY_1,MUNICIPALITY_2,MUNICIPALITY_3,MUNICIPALITY_4
0,0,0,59,2,0,1.4583,0.00,12.67,251903,3.281095,...,0,0,0,0,0,0,0,0,1,0
1,0,0,59,2,0,2.2157,0.18,13.67,277590,3.169154,...,0,0,0,0,0,0,0,0,1,0
2,0,0,68,3,1,2.2157,0.32,14.29,296572,3.002463,...,0,0,0,0,0,0,0,0,1,0
3,0,0,31,1,1,2.2157,0.00,11.67,221745,3.845905,...,0,0,0,0,0,0,0,0,1,0
4,0,0,79,4,0,2.2157,0.48,13.46,564137,3.002463,...,0,0,0,0,0,0,0,0,1,0


In [48]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Target'])],
                                                    df.loc[:, df.columns == 'Target'], shuffle=True, test_size=0.25,
                                                    random_state=1, stratify=df.loc[:, df.columns == 'Target'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#X_train.to_pickle('x_train_lvl3.pkl')
#X_test.to_pickle('x_test_lvl3.pkl')
#y_train.to_pickle('y_train_lvl3.pkl')
#y_test.to_pickle('y_test_lvl3.pkl')

(122067, 67) (40690, 67) (122067, 1) (40690, 1)


# Lvl 4 (bank)

In [46]:
df = pd.get_dummies(df, columns=['CL_age','CL_accom', 'Employment'])
df['Salary'] = df.Salary.astype(int)
df = df.rename(columns={"gbflag": "Target"})

In [47]:
df.head()

,Target,Salary,CL_age_27-36,CL_age_37-47,CL_age_48-61,CL_age_62-High,CL_age_LOW-26,CL_accom_Ejer,CL_accom_Lejer,CL_accom_Other,Employment_Arbejdsløs,Employment_Efterløn,Employment_Funktionær,Employment_Hjemmegående,Employment_Ledelse,Employment_Other,Employment_Pensionist,Employment_Selvstændig,Employment_Studerende,Employment_Timelønnet
0,0,15000,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0,13100,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
2,0,12000,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
3,0,13000,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,10288,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [48]:
df.head()

,Target,Salary,CL_age_27-36,CL_age_37-47,CL_age_48-61,CL_age_62-High,CL_age_LOW-26,CL_accom_Ejer,CL_accom_Lejer,CL_accom_Other,Employment_Arbejdsløs,Employment_Efterløn,Employment_Funktionær,Employment_Hjemmegående,Employment_Ledelse,Employment_Other,Employment_Pensionist,Employment_Selvstændig,Employment_Studerende,Employment_Timelønnet
0,0,15000,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0,13100,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
2,0,12000,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
3,0,13000,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,10288,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [49]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Target'])],
                                                    df.loc[:, df.columns == 'Target'], shuffle=True, test_size=0.2,
                                                    random_state=1, stratify=df.loc[:, df.columns == 'Target'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train.to_pickle('x_train_lvl4.pkl')
X_test.to_pickle('x_test_lvl4.pkl')
y_train.to_pickle('y_train_lvl4.pkl')
y_test.to_pickle('y_test_lvl4.pkl')

(31796, 19) (7950, 19) (31796, 1) (7950, 1)


EOFError: Ran out of input

In [49]:
X_train.to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\X_train_lvl7.csv', index=False)
pd.DataFrame(y_train).to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\y_train_lvl7.csv', index=False)
X_test.to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\X_test_lvl7.csv', index=False)
pd.DataFrame(y_test).to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\y_test_lvl7.csv', index=False)

# Lvl 5 - like lvl 3 but with no outliers

In [62]:
df

,HOU_HOLHOMOWNR,OWNSHIP,AREA,ROOMS,BUSINESS_ON_ADRESS,PER_COU_UNADR,CAR_AVG,EDU_AVG,HOU_INCO_AVG,SOCGRP_AVG,...,CNZTYP_H1,CNZTYP_H2,CNZTYP_I1,CNZTYP_I2,CNZTYP_I3,CNZTYP_I4,MUNICIPALITY_1,MUNICIPALITY_2,MUNICIPALITY_3,MUNICIPALITY_4
0,0,0,59,2,0,1.4583,0.00,12.67,251903,3.281095,...,0,0,0,0,0,0,0,0,1,0
1,0,0,59,2,0,2.2157,0.18,13.67,277590,3.169154,...,0,0,0,0,0,0,0,0,1,0
2,0,0,68,3,1,2.2157,0.32,14.29,296572,3.002463,...,0,0,0,0,0,0,0,0,1,0
3,0,0,31,1,1,2.2157,0.00,11.67,221745,3.845905,...,0,0,0,0,0,0,0,0,1,0
4,0,0,79,4,0,2.2157,0.48,13.46,564137,3.002463,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163127,0,0,134,5,0,1.4412,0.74,15.67,702901,2.803483,...,0,0,0,0,0,0,0,0,1,0
163128,0,1,91,4,0,1.7241,0.36,16.33,543046,2.125926,...,0,0,0,0,0,0,0,0,1,0
163129,0,1,160,5,0,2.4286,1.16,13.51,902323,3.270073,...,0,0,0,0,0,0,1,0,0,0
163130,0,0,77,2,0,1.4872,0.36,12.33,299211,3.827196,...,0,0,0,0,0,0,0,1,0,0


In [154]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Target'])],
                                                    df.loc[:, df.columns == 'Target'], shuffle=True, test_size=0.25,
                                                    random_state=1, stratify=df.loc[:, df.columns == 'Target'])

In [139]:
def find_outliers_IQR(df):
    q1 = df.quantile(0.25)

    q3 = df.quantile(0.75)

    IQR = q3 - q1

    outliers = df[((df < (q1 - 1.5 * IQR)) | (df > (q3 + 1.5 * IQR)))]

    return outliers

In [158]:
print(X_train.shape, y_train.shape)
X_train_1 = X_train
X_train_1["Target"] = y_train
X_train_1.shape

(122067, 67) (122067, 1)


(122067, 67)

In [161]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122067 entries, 79091 to 43259
Data columns (total 67 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   HOU_HOLHOMOWNR         122067 non-null  int32  
 1   OWNSHIP                122067 non-null  int32  
 2   AREA                   122067 non-null  int32  
 3   ROOMS                  122067 non-null  int32  
 4   BUSINESS_ON_ADRESS     122067 non-null  int32  
 5   PER_COU_UNADR          122067 non-null  float64
 6   CAR_AVG                122067 non-null  float64
 7   EDU_AVG                122067 non-null  float64
 8   HOU_INCO_AVG           122067 non-null  int32  
 9   SOCGRP_AVG             122067 non-null  float64
 10  WEALTH_AVG             122067 non-null  int32  
 11  AGE_OLDST_AVG          122067 non-null  float64
 12  CHL_AVG                122067 non-null  float64
 13  PER_INCO_AVG2          122067 non-null  float64
 14  EMPL_STATUS            122067 non

In [162]:
outliers= find_outliers_IQR(X_train["WEALTH_AVG"][X_train["Target"]==0]) #bad customers that should be good
print("number of outliers: " + str(len(outliers)))
print('max outerlier value: ' + str(outliers.max()))
print('min outlier value: ' + str(outliers.min()))
print(outliers)

number of outliers: 2057
max outerlier value: 9605061
min outlier value: -2315725
33451     3245003
55578     1844175
11970     3689269
49938     3940824
72722     2653433
           ...   
18613     1960252
90424     3200312
139898    3288013
97905     7199750
121759    4703930
Name: WEALTH_AVG, Length: 2057, dtype: int32


In [67]:
outliers = find_outliers_IQR(df["WEALTH_AVG"][df["Target"]==1]) #good customers that should be bad
print("number of outliers: " + str(len(outliers)))
print('max outerlier value: ' + str(outliers.max()))
print('min outlier value: ' + str(outliers.min()))
print(outliers)

number of outliers: 6539
max outerlier value: 9860191
min outlier value: 4339148
25        5003827
30        6069692
31        6069692
32        6759109
33        6759109
           ...   
162922    4981052
162990    4339148
163006    4482410
163018    5063835
163040    5791765
Name: WEALTH_AVG, Length: 6539, dtype: int32


In [73]:
X_train["WEALTH_AVG"][X_train["Target"]==1].mean()

1156507.053326703

In [72]:
outliers[outliers<214749]

Series([], Name: WEALTH_AVG, dtype: int32)

In [163]:
outliers_min = outliers[outliers>0]
outliers_min

33451     3245003
55578     1844175
11970     3689269
49938     3940824
72722     2653433
           ...   
18613     1960252
90424     3200312
139898    3288013
97905     7199750
121759    4703930
Name: WEALTH_AVG, Length: 2020, dtype: int32

In [165]:
X_train.shape

(122067, 67)

In [168]:
X_train_1 = X_train_1.drop(index=outliers_min.index, axis=0)
X_train_1

,HOU_HOLHOMOWNR,OWNSHIP,AREA,ROOMS,BUSINESS_ON_ADRESS,PER_COU_UNADR,CAR_AVG,EDU_AVG,HOU_INCO_AVG,SOCGRP_AVG,...,CNZTYP_H2,CNZTYP_I1,CNZTYP_I2,CNZTYP_I3,CNZTYP_I4,MUNICIPALITY_1,MUNICIPALITY_2,MUNICIPALITY_3,MUNICIPALITY_4,Target
79091,0,1,236,5,0,2.2500,1.07,14.67,1063442,2.204938,...,0,0,0,0,0,0,1,0,0,1
85628,0,1,120,5,0,2.1196,1.55,15.00,925847,2.706468,...,0,0,0,0,0,0,1,0,0,1
128366,0,0,56,1,0,1.2266,0.19,16.67,568765,2.430348,...,0,0,0,0,0,0,0,1,0,1
56280,0,1,159,4,1,1.5000,1.53,14.16,775615,2.883951,...,0,0,0,0,0,0,0,1,0,1
149077,0,0,62,2,0,0.6782,0.34,12.00,265705,3.534826,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72396,0,0,96,3,0,2.1982,0.56,11.30,406098,3.684939,...,0,0,0,0,0,0,1,0,0,0
33871,0,0,96,3,0,1.5556,0.75,11.45,393734,2.948020,...,0,0,1,0,0,0,1,0,0,1
121996,0,1,97,4,0,1.9676,1.10,14.00,775201,2.558025,...,0,0,0,0,0,0,0,0,1,1
137523,0,0,67,3,0,1.5385,0.56,14.33,370631,2.840796,...,0,0,0,0,0,0,0,1,0,1


In [169]:

y_train = X_train_1["Target"]
X_train = X_train_1.drop("Target", axis=1)
print(y_train.shape, X_train.shape)

(120047,) (120047, 66)


In [170]:
#X_train, X_test, y_train, y_test = train_test_split(X_train.loc[:, ~X_train.columns.isin(['Target'])],
 #                                                   X_train.loc[:, X_train.columns == 'Target'], shuffle=True, test_size=0.2,
  #                                                  random_state=1, stratify=X_train.loc[:, X_train.columns == 'Target'])
#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train.to_pickle('x_train_lvl5.pkl')
X_test.to_pickle('x_test_lvl5.pkl')
y_train.to_pickle('y_train_lvl5.pkl')
y_test.to_pickle('y_test_lvl5.pkl')

In [171]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(120047, 66) (40690, 66) (120047,) (40690, 1)


In [172]:
X_train.to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\X_train_lvl5.csv', index=False)
pd.DataFrame(y_train).to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\y_train_lvl5.csv', index=False)
X_test.to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\X_test_lvl5.csv', index=False)
pd.DataFrame(y_test).to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\y_test_lvl5.csv', index=False)

# Level 6 - many variables

In [91]:
df['HOU_HOLHOMOWNR'] = (df['HOU_HOLHOMOWNR'] == True).astype(int)
df['BUSINESS_ON_ADRESS'] = (df['BUSINESS_ON_ADRESS'] == True).astype(int)
df['Target'] = df.Target.astype(int)
df = df.drop(['CNZGRP', 'UNADR_KEY'], axis=1)
#df = employment(df)
#df = child(df)
#df = mari_status(df)
df = ownship(df)
df = muni(df)

[3. 2. 4. 3. 1. 1. 2. 3. 2. 4. 3. 3. 2. 3. 4. 2. 2. 2. 1. 2. 3. 2. 1. 2.
 1. 2. 2. 3. 3. 2. 1. 1. 3. 2. 3. 3. 3. 4. 3. 4. 1. 4. 4. 4. 3. 3. 3. 2.
 2. 2. 3. 3. 3. 2. 3. 4. 4. 2. 3. 4. 3. 1. 3. 2. 4. 3. 2. 1. 1. 1. 3. 4.
 4. 2. 2. 1. 1. 3. 3. 2. 1. 2. 1. 2. 1. 4. 3. 2. 3. 1. 2. 4. 3. 1. 3. 2.
 2. 2.]


In [92]:
df = df.loc[~((df['ROOMS'] > 9) & (df['USAGE'] != 'Farmhouse') & (df['USAGE'] != 'Institution') & (
            df['BUSINESS_ON_ADRESS'] == False))]
df = pd.get_dummies(df, columns=['USAGE', 'CNZTYP', 'MUNICIPALITY', 'HOU_HABITATION_CODE'])
df['AREA'] = df.AREA.astype(int)
df['ROOMS'] = df.ROOMS.astype(int)
#df['HOU_INCO_AVG'] = df.HOU_INCO_AVG.astype(int)
#df['WEALTH_AVG'] = df.WEALTH_AVG.astype(int)
#df['EMPL_STATUS'] = df.EMPL_STATUS.astype(int)
#df['MARI_STATUS'] = df.MARI_STATUS.astype(int)
df['OWNSHIP'] = df.OWNSHIP.astype(int)
#df['PER_INCO_AVG2'] = df.OWNSHIP.astype(int)
df['HOU_HOLHOMOWNR'] = (df['HOU_HOLHOMOWNR'] == True).astype(int)
df['BUSINESS_ON_ADRESS'] = (df['BUSINESS_ON_ADRESS'] == True).astype(int)
df['Target'] = df.Target.astype(int)
df.head()

,HOU_HOLHOMOWNR,OWNSHIP,AREA,ROOMS,BUSINESS_ON_ADRESS,PER_COU_UNADR,HOU_LIFEPHASE_V2_DIS_UNADR_YOUNG,HOU_LIFEPHASE_V2_DIS_UNADR_STARTERS_SINGLE,HOU_LIFEPHASE_V2_DIS_UNADR_STARTERS_COUPLE,HOU_LIFEPHASE_V2_DIS_UNADR_SETTLED_SINGLE,...,MUNICIPALITY_1,MUNICIPALITY_2,MUNICIPALITY_3,MUNICIPALITY_4,HOU_HABITATION_CODE_CITY,HOU_HABITATION_CODE_COUNTRY,HOU_HABITATION_CODE_LARGETOWN,HOU_HABITATION_CODE_SMALLTOWN,HOU_HABITATION_CODE_SUBURB,HOU_HABITATION_CODE_VILLAGE
0,0,0,59,2,0,1.4583,0.000000,0.016114,0.010396,0.301552,...,0,0,1,0,1,0,0,0,0,0
1,0,0,59,2,0,2.2157,0.000000,0.010015,0.004871,0.322162,...,0,0,1,0,1,0,0,0,0,0
2,0,0,68,3,1,2.2157,0.002475,0.012437,0.017193,0.231332,...,0,0,1,0,1,0,0,0,0,0
3,0,0,31,1,1,2.2157,0.012285,0.041321,0.020096,0.494195,...,0,0,1,0,1,0,0,0,0,0
4,0,0,79,4,0,2.2157,0.019851,0.012505,0.022724,0.168814,...,0,0,1,0,1,0,0,0,0,0


In [99]:
df.describe()

,HOU_HOLHOMOWNR,OWNSHIP,AREA,ROOMS,BUSINESS_ON_ADRESS,PER_COU_UNADR,HOU_LIFEPHASE_V2_DIS_UNADR_YOUNG,HOU_LIFEPHASE_V2_DIS_UNADR_STARTERS_SINGLE,HOU_LIFEPHASE_V2_DIS_UNADR_STARTERS_COUPLE,HOU_LIFEPHASE_V2_DIS_UNADR_SETTLED_SINGLE,...,MUNICIPALITY_1,MUNICIPALITY_2,MUNICIPALITY_3,MUNICIPALITY_4,HOU_HABITATION_CODE_CITY,HOU_HABITATION_CODE_COUNTRY,HOU_HABITATION_CODE_LARGETOWN,HOU_HABITATION_CODE_SMALLTOWN,HOU_HABITATION_CODE_SUBURB,HOU_HABITATION_CODE_VILLAGE
count,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,...,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000,162757.000000
mean,0.052649,0.579686,111.777091,3.890819,0.144455,2.112972,0.015586,0.025900,0.023870,0.208642,...,0.131097,0.325977,0.422919,0.120007,0.136424,0.101575,0.313885,0.141677,0.196618,0.109820
std,0.223333,0.493611,52.322117,4.600125,0.351551,0.916173,0.033739,0.055056,0.036295,0.151987,...,0.337508,0.468740,0.494024,0.324971,0.343240,0.302089,0.464072,0.348720,0.397442,0.312666
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.017200,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,79.000000,3.000000,0.000000,1.613100,0.000000,0.002785,0.004847,0.072313,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,100.000000,4.000000,0.000000,2.000000,0.005000,0.007885,0.011380,0.189216,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,136.000000,5.000000,0.000000,2.500000,0.017241,0.021723,0.026573,0.318387,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,2603.000000,999.000000,1.000000,102.000000,0.778824,0.531560,0.394269,0.672860,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [101]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Target'])],
                                                    df.loc[:, df.columns == 'Target'], shuffle=True, test_size=0.25,
                                                    random_state=1, stratify=df.loc[:, df.columns == 'Target'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train.to_pickle('x_train_lvl6.pkl')
X_test.to_pickle('x_test_lvl6.pkl')
y_train.to_pickle('y_train_lvl6.pkl')
y_test.to_pickle('y_test_lvl6.pkl')

(122067, 152) (40690, 152) (122067, 1) (40690, 1)


Highest and lowest wealth clusters within Copenhagen

In [23]:
df['HOU_HOLHOMOWNR'] = (df['HOU_HOLHOMOWNR'] == True).astype(int)
df['BUSINESS_ON_ADRESS'] = (df['BUSINESS_ON_ADRESS'] == True).astype(int)
df['Target'] = df.Target.astype(int)
df = employment(df)
df = child(df)
df = mari_status(df)
df = ownship(df)
#df = muni(df)


In [17]:
df.head()

,UNADR_KEY,HOU_HOLHOMOWNR,MUNICIPALITY,CNZTYP,OWNSHIP,USAGE,AREA,ROOMS,BUSINESS_ON_ADRESS,CAR_AVG,EDU_AVG,HOU_INCO_AVG,SOCGRP_AVG,WEALTH_AVG,AGE_OLDST_AVG,CHL_AVG,Target,UNADR_GMC_WEALTH_V2_2201_KEY,EMPL_STATUS,MARI_STATUS
0,22318638,0,101,F1,1,Apartment,173,5,0,0.38,18.00,2789763,1.217770,7167068,62.00,0.17,0,101000001,1.0,0.0
1,22354765,1,101,F2,1,Apartment,218,6,1,0.78,17.33,4128136,1.217770,8277951,56.67,1.67,0,101000001,1.0,1.0
2,22406775,0,101,F1,1,Apartment,148,4,1,0.77,17.67,1216560,2.000000,4568241,61.00,0.17,1,101000001,1.0,1.0
3,22435422,1,101,F1,1,Apartment,175,4,1,0.99,15.61,2359663,1.651741,7758345,71.37,0.17,0,101000001,0.0,0.0
4,22500012,1,101,F1,1,Apartment,287,9,0,0.96,17.00,3155869,1.217770,9246625,66.37,0.17,0,101000001,1.0,0.0


In [24]:
df['AREA'] = df.AREA.astype(int)
df['ROOMS'] = df.ROOMS.astype(int)
df['HOU_INCO_AVG'] = df.HOU_INCO_AVG.astype(int)
df['WEALTH_AVG'] = df.WEALTH_AVG.astype(int)
df['EMPL_STATUS'] = df.EMPL_STATUS.astype(int)
df['MARI_STATUS'] = df.MARI_STATUS.astype(int)
df['OWNSHIP'] = df.OWNSHIP.astype(int)
df['HOU_HOLHOMOWNR'] = (df['HOU_HOLHOMOWNR'] == True).astype(int)
df['BUSINESS_ON_ADRESS'] = (df['BUSINESS_ON_ADRESS'] == True).astype(int)
df['Target'] = df.Target.astype(int)
df.head()

,UNADR_KEY,HOU_HOLHOMOWNR,MUNICIPALITY,CNZTYP,OWNSHIP,USAGE,AREA,ROOMS,BUSINESS_ON_ADRESS,CAR_AVG,EDU_AVG,HOU_INCO_AVG,SOCGRP_AVG,WEALTH_AVG,AGE_OLDST_AVG,CHL_AVG,Target,UNADR_GMC_WEALTH_V2_2101_KEY,EMPL_STATUS,MARI_STATUS
0,22310021,0,101,D3,1,Attached houses,138,4,0,0.99,17.64,823587,1.626866,9246625,74.06,0.00,0,101000001,1,1
1,22373364,0,101,F1,1,Apartment,316,8,0,1.23,17.33,3155869,1.217770,9246625,62.84,0.17,1,101000001,1,1
2,22373366,1,101,F1,1,Apartment,306,8,0,1.23,17.33,3155869,1.217770,9246625,62.84,0.17,0,101000001,1,1
3,22414614,0,101,F1,1,Apartment,227,4,1,0.75,17.67,2789763,1.217770,9246625,56.33,0.33,0,101000001,1,0
4,22470624,0,101,D1,1,Villa,224,8,1,1.23,17.67,4128136,1.217770,9246625,50.67,1.83,1,101000001,1,0


In [25]:
df.to_csv(r'\\geo-file\geomatic\Temp\mao\Thesis\Data\df_high_low_wealth.csv', index=False)
